In [1]:
# 必要なライブラリのインポート
from bs4 import BeautifulSoup
import pandas as pd
import requests
import datetime as dt
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import get_as_dataframe, set_with_dataframe

# パスの取得に必要なリストの定義
new_path = []
path = []

# データフレームの作成に必要なリストの定義
update_date_list = []
house_image_list = []
address_list = []
distance_list = []
year_list = []
level_list = []
room_image_list = []
rent_price_list = []
admin_price_list = []
deposit_price_list = []
key_price_list = []
size_list = []
area_list = []
url_list = []
favolite_list = []

# 今日の日付
today = format(dt.datetime.today(),'%Y/%m/%d')

# SUUMOの検索ページ（指定済みURL）
url = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ra=014&cb=0.0&ct=10.0&co=1&et=15&md=09&md=10&md=11&md=12&md=13&md=14&cn=9999999&mb=45&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&ek=016026210&rn=0160'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

# 住宅ごと
elems = soup.find_all('div', class_='cassetteitem')
for index, elem in enumerate(elems):

    # 部屋ごと
    elems_room = elem.find_all('tr', class_='js-cassette_link')
    for elem_room in elems_room:

        # 更新日
        update_date_list.append(today)

        # 家の画像
        house_image_list.append('=IMAGE("' + elem.find('img', class_='js-linkImage')['rel'] + '")')

        # 住所
        address_list.append(elem.find('li', class_='cassetteitem_detail-col1').text)

        # 駅徒歩
        distance_list.append(elem.find('div', class_='cassetteitem_detail-text').text)

        # 築年数と階層
        elem1 = elem.find('li', class_='cassetteitem_detail-col3')
        # 築年数
        year_list.append(elem1.find_all('div')[0].text)
        # 階層
        level_list.append(elem1.find_all('div')[1].text)
        
        # 間取り図
        room_image_list.append('=IMAGE("' + elem_room.find('img')['rel'] + '")')

        # 賃料
        rent_price_list.append(elem_room.find('span', class_='cassetteitem_other-emphasis').text)

        # 管理費
        admin_price_list.append(elem_room.find('span', class_='cassetteitem_price--administration').text)

        # 敷金
        deposit_price_list.append(elem_room.find('span', class_='cassetteitem_price--deposit').text)

        # 礼金
        key_price_list.append(elem_room.find('span', class_='cassetteitem_price--gratuity').text)

        # 間取り
        size_list.append(elem_room.find('span', class_='cassetteitem_madori').text)
        
        # 占有面積
        area_list.append(elem_room.find('span', class_='cassetteitem_menseki').text)

        # 詳細リンク取得
        url_list.append('https://suumo.jp' + elem_room.find('a', class_='js-cassette_link_href')['href'])

        # お気に入り登録
        favolite_list.append(False)

# データフレーム作成
df = pd.DataFrame()

df['更新日'] = update_date_list
df['家の写真'] = house_image_list
df['住所'] = address_list
df['駅徒歩'] = distance_list
df['築年数'] = year_list
df['階層'] = level_list
df['間取り図'] = room_image_list
df['賃料'] = rent_price_list
df['管理費'] = admin_price_list
df['敷金'] = deposit_price_list
df['礼金'] = key_price_list
df['間取り'] = size_list
df['占有面積'] = area_list
df['詳細リンク'] = url_list
df['お気に入り'] = favolite_list

# CSVファイル出力
# df.to_csv(f'./output/SUUMO.csv', index=False)

# スプレッドシート連携
# creds を使って Google Drive API と対話するためのクライアントを作成
scope =['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name(r'./python-spread.json', scope)
client = gspread.authorize(creds)

# ワークブックを検索、最初のシートを開く
worksheet = client.open("SUUMOスクレイピング").sheet1

# すべての値を取得
list_of_hashes = worksheet.get_all_records()

# スプレッドシートのデータフレーム作成
df_sp = pd.DataFrame()
df_sp = pd.DataFrame(list_of_hashes)

# 帝国スクレイピングとスプシのデータ統合
df_con = pd.concat([df,df_sp])

# スプレッドシートのデータ更新

set_with_dataframe(worksheet, df_con.reset_index(drop=True))

print('更新完了')

更新完了


In [3]:

# スプレッドシート連携
# creds を使って Google Drive API と対話するためのクライアントを作成
scope =['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name(r'./python-spread.json', scope)
client = gspread.authorize(creds)

# ワークブックを検索、最初のシートを開く
worksheet = client.open("SUUMOスクレイピング").sheet1

# すべての値を取得
list_of_hashes = worksheet.get_all_records()

# スプレッドシートのデータフレーム作成
df_sp = pd.DataFrame()
df_sp = pd.DataFrame(list_of_hashes)

# 帝国スクレイピングとスプシのデータ統合
df_con = pd.concat([df,df_sp])

# スプレッドシートのデータ更新

set_with_dataframe(worksheet, df_con.reset_index(drop=True))

print('更新完了')

更新完了


In [16]:
import datetime as dt
format(dt.datetime.today(),'%Y/%m/%d')

'2023/03/12'

In [20]:
elem.find('img', class_='js-linkImage')[]

<img alt="" class="js-noContextMenu js-linkImage js-scrollLazy js-adjustImg" height="0" rel="https://img01.suumo.com/front/gazo/fr/bukken/332/100320004332/100320004332_gw.jpg" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAAAAACH5BAEAAAAALAAAAAABAAEAAAICRAEAOw%3D%3D" width="0">
</img>